In [ ]:
import beneath
import plotly.express as px

from config import BLACKLIST, MODERATORS
from layout import make_layout

In [ ]:
# July report
MONTH_NUMBER=7

## Stock mentions

In [ ]:
stocks = await beneath.query_warehouse("""
with
    stock_mentions_posts as (
        select 
            symbol, 
            timestamp_trunc(timestamp, day) as day, 
            count(*) as num_mentions
        from `examples/wallstreetbets-analytics/r-wallstreetbets-posts-stock-mentions`
        group by symbol, timestamp_trunc(timestamp, day)
    ),
    stock_mentions_comments as (
        select 
            symbol, 
            timestamp_trunc(timestamp, day) as day, 
            count(*) as num_mentions
        from `examples/wallstreetbets-analytics/r-wallstreetbets-comments-stock-mentions`
        group by symbol, timestamp_trunc(timestamp, day)
    ),
    mentions as (
        select 
            coalesce(p.symbol, c.symbol) as symbol,
            coalesce(p.day, c.day) as day,
            ifnull(p.num_mentions, 0) + ifnull(c.num_mentions,0) as num_mentions
        from stock_mentions_posts p
        full join stock_mentions_comments c on p.symbol = c.symbol and p.day = c.day
    )
select m.symbol, m.day, m.num_mentions, p.close
from mentions m
left join `examples/wallstreetbets-analytics/stock_prices_daily` p on m.symbol = p.symbol and m.day = p.date
order by symbol, day
""")

A bit of feature engineering

In [ ]:
# filter out blacklist
stocks = stocks[~stocks['symbol'].isin(BLACKLIST)]

# get the fraction of all mentions in the day (important to do this after applying the blacklist)
mentions_daily = stocks.groupby('day')['num_mentions'].sum().reset_index()
stocks = stocks \
    .merge(mentions_daily, on="day", suffixes=('_stock', '_total')) \
    .sort_values(['symbol','day'])
stocks['fraction_of_mentions'] = stocks['num_mentions_stock'] / stocks['num_mentions_total']
stocks['fraction_of_mentions_MA'] = stocks.groupby('symbol')['fraction_of_mentions'] \
                                    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# get the date of peak popularity
symbol_peaks = stocks.loc[stocks.groupby(['symbol'])["fraction_of_mentions_MA"].idxmax()][['symbol', 'day']] \
    .rename(columns={'day': 'date_of_peak_popularity'})
stocks = stocks.merge(symbol_peaks, on='symbol')
stocks['days_from_peak'] = (stocks['day'] - stocks['date_of_peak_popularity']).dt.days

# keep track of the top symbols
top_symbols_alltime_df = stocks \
    .groupby('symbol')['num_mentions_stock'] \
    .sum() \
    .reset_index() \
    .sort_values('num_mentions_stock', ascending=False)
top_symbols_alltime = top_symbols_alltime_df['symbol'][:500]
top_symbols_month_df = stocks.loc[stocks['day'].dt.month == MONTH_NUMBER] \
    .groupby('symbol')['num_mentions_stock'] \
    .sum() \
    .reset_index() \
    .sort_values('num_mentions_stock', ascending=False)
top_symbols_month = top_symbols_month_df['symbol'][:100]

In [ ]:
# save top symbols
# I'll allow these symbols to be searchable in my wsb dashboard
top_symbols_alltime_df[:500].to_csv('/workspaces/beneath/examples/wallstreetbets-analytics/explore/data/top_symbols_alltime.csv')

In [ ]:
# assign colors to symbols used in blogpost, for consistency across graphs
chinese_symbols = ['BABA', 'JD', 'PDD', 'TAL']
symbols_in_blogpost = top_symbols_month[0:10].tolist() + chinese_symbols
colors = px.colors.qualitative.Plotly
blogpost_colors = dict(zip(symbols_in_blogpost, colors))

In [ ]:
fig = px.bar(
    top_symbols_month_df[0:10],
    x="num_mentions_stock",
    y="symbol",
    text="num_mentions_stock",
    color='symbol',
    color_discrete_map=blogpost_colors,
    orientation='h',
    labels={'num_mentions_stock': 'Mentions', 'symbol': 'Symbol'},
)
fig.update_layout(
    make_layout(
        title="Top 10 Most Popular Symbols in July",
        subtitle="Number of mentions in posts and comments",
        legend=False,
        source_hidden=False,
        override={
            "bargap": 0,
            'showlegend': False,
            'yaxis_title': '',
            'xaxis_zeroline': False
        },
    )
)
fig.update_traces({
    'marker_line_width': 0                 
})
fig.show()

In [ ]:
fig = px.line(stocks.loc[(stocks['symbol'].isin(top_symbols_month[:10])) & (stocks['day'].dt.month == MONTH_NUMBER)], 
              x="day",
              y="num_mentions_stock",
              line_group="symbol",
              color="symbol",
              color_discrete_map=blogpost_colors,
              labels={'num_mentions_stock': 'Mentions', 'day': 'Day', 'symbol': 'Symbol'},
             )
fig.update_layout(
    make_layout(
        title="Stock mentions in July",
        subtitle="Number of mentions in posts and comments",
        source_hidden=False,
        legend=True,
        override={
            'xaxis': {
#                 'tickmode': 'array',
#                 'tickvals': ["2021-07-07", "2021-07-14", "2021-07-21", "2021-07-28"],
                'title': ''
            }
        },
    )
)
fig.show()

Relative interest in GME and AMC has declined, following their big price spikes

In [ ]:
symbols_to_graph = ['AMC', 'GME']
fig = px.line(stocks[stocks['symbol'].isin(symbols_to_graph)].sort_values('day', ascending=False), 
              x="day",
              y="fraction_of_mentions", 
              line_group="symbol",
              color="symbol",
              color_discrete_map=blogpost_colors,
              labels={'day': 'Day', 'fraction_of_mentions': 'Fraction of all mentions', 'symbol': 'Symbol'},
            )
fig.update_layout(
    make_layout(
        title="Relative interest in GME and AMC",
        subtitle="Share of mentions across all stocks on r/wallstreetbets",
        source_hidden=False,
        legend=True,
        override={
            'xaxis': {
                'title': ''
            }
        },
    )
)
fig.show()

In [ ]:
symbols_to_graph = ['CLOV', 'SPCE', 'WISH']
fig = px.line(stocks[stocks['symbol'].isin(symbols_to_graph)].sort_values('day', ascending=False), 
              x="day",
              y="fraction_of_mentions", 
              line_group="symbol",
              color="symbol",
              color_discrete_map=blogpost_colors,
              labels={'day': 'Day', 'fraction_of_mentions': 'Fraction of mentions', 'symbol': 'Symbol'},
            )
fig.update_layout(
    make_layout(
        title="Relative interest in CLOV and SPCE has increased",
        subtitle="Share of mentions across all stocks on r/wallstreetbets",
        source_hidden=False,
        legend=True,
        override={
            'xaxis': {
                'title': ''
            }
        },
    )
)
fig.show()

Find stocks that are peaking

In [ ]:
# JD, BABA, TAL, PDD
stocks.loc[(stocks['day'] == "2021-07-27") & 
             (stocks['days_from_peak'] < 10) & 
             (stocks['num_mentions_stock'] > 10) ]

In [ ]:
blogpost_colors

In [ ]:
symbols_to_graph = ['JD', 'BABA', 'TAL', 'PDD']
fig = px.line(stocks[(stocks['symbol'].isin(symbols_to_graph)) & (stocks['day'] >= '2021-07-01')], 
              x="day",
              y="fraction_of_mentions", 
              line_group="symbol",
              color="symbol",
              color_discrete_map=blogpost_colors,
              labels={'day': 'Day', 'fraction_of_mentions': 'Fraction of mentions', 'symbol': 'Symbol'},
            )
fig.update_layout(
    make_layout(
        title="Chinese stocks were buzzing at the end of July",
        subtitle="Share of mentions across all stocks on r/wallstreetbets",
        source_hidden=False,
        legend=True,
        override={
            'xaxis': {
                'title': ''
            }
        },
    )
)
fig.show()

In [ ]:
symbols_to_plot = top_symbols_alltime[0:10]
fig = px.line(stocks[stocks['symbol'].isin(symbols_to_plot)].sort_values('day', ascending=False), 
              x="days_from_peak", 
              y="num_mentions_stock",
              line_group="symbol",
              color="symbol",
              labels={'days_from_peak': 'Days from peak', 'num_mentions_stock': 'Mentions', 'symbol': 'Symbol'},
             )
fig.update_layout(
    make_layout(
        title="Days from peak",
        subtitle="...",
        source_hidden=False,
        legend=True,
        override={},
    )
)
fig.show()

## Interesting discussions

In [ ]:
pd.set_option('max_colwidth', 100)

In [ ]:
posts = await beneath.query_warehouse("""
with
    posts as (
        select 
            created_on, 
            id, 
            author, 
            title, 
            text, 
            flair, 
            permalink,
            length(text) as post_length,
            array_length(regexp_extract_all(title, r"\\x{1F680}")) + array_length(regexp_extract_all(text, r"\\x{1F680}")) as num_rockets,
            array_length(regexp_extract_all(title, r"\\x{1F48E}")) + array_length(regexp_extract_all(text, r"\\x{1F48E}")) as num_diamonds,
        from `examples/reddit/r-wallstreetbets-posts`
        where timestamp_trunc(created_on, month) = "2021-07-01"
    ),
    comments as (
        select 
            created_on, 
            id, 
            post_id, 
            author, 
            text,
            length(text) as comment_length,
            array_length(regexp_extract_all(c.text, r"\\x{1F680}")) as num_rockets,
            array_length(regexp_extract_all(c.text, r"\\x{1F48E}")) as num_diamonds,
        from `examples/reddit/r-wallstreetbets-comments` c
        where timestamp_trunc(created_on, month) = "2021-07-01"
    )
select 
    p.created_on, 
    p.title, 
    p.text, 
    p.author, 
    p.flair, 
    p.permalink,
    p.post_length,
    count(c.id) as num_comments,
    sum(c.comment_length) as sum_comments_length, 
    sum(c.comment_length)/count(c.id) as avg_comment_length,
    p.num_rockets + sum(c.num_rockets) as num_rockets,
    p.num_diamonds + sum(c.num_diamonds) as num_diamonds,
    count(distinct c.author) as nunique_commenters,
from posts p
join comments c on p.id = c.post_id
group by p.created_on, p.title, p.text, p.author, p.flair, p.permalink, p.post_length, p.num_rockets, p.num_diamonds
""")

Exclude moderators

In [ ]:
posts_no_mods = posts.loc[[author not in MODERATORS for author in posts['author']]]

Post with most comments

In [ ]:
posts_no_mods.sort_values('num_comments', ascending=False).head(3)

Longest discussion

In [ ]:
posts_no_mods.sort_values('sum_comments_length', ascending=False).head(3)

Post with the highest avg comment length (at least 5 comments)

In [ ]:
posts_no_mods[posts_no_mods['num_comments'] >= 15].sort_values('avg_comment_length', ascending=False).head(3)

Post with the most rockets (including comments)

In [ ]:
posts_no_mods.sort_values('num_rockets', ascending=False).head(3)

Post with the most diamonds (including comments)

In [ ]:
posts_no_mods.sort_values('num_diamonds', ascending=False).head(3)

## Degenerate shoutouts

In [ ]:
authors = await beneath.query_warehouse("""
with
    posts_enhanced as (
        select *,
            array_length(split(title, " ")) as num_words_title,
            array_length(split(text, " ")) as num_words_body,
            array_length(regexp_extract_all(title, r"\\x{1F680}")) as num_rockets_title,
            array_length(regexp_extract_all(text, r"\\x{1F680}")) as num_rockets_body,
            array_length(regexp_extract_all(title, r"\\x{1F48E}")) as num_diamonds_title,
            array_length(regexp_extract_all(text, r"\\x{1F48E}")) as num_diamonds_body
        from `examples/reddit/r-wallstreetbets-posts`
        where timestamp_trunc(created_on, month) = "2021-07-01"
    ),
    comments_enhanced as (
        select *,
            array_length(split(text, " ")) as num_words,
            array_length(regexp_extract_all(text, r"\\x{1F680}")) as num_rockets,
            array_length(regexp_extract_all(text, r"\\x{1F48E}")) as num_diamonds
        from `examples/reddit/r-wallstreetbets-comments`
        where timestamp_trunc(created_on, month) = "2021-07-01"
    ),
    author_posts_stats as (
        select 
            author, 
            count(*) as num_posts,
            sum(num_words_title) + sum(num_words_body) as num_words,
            sum(num_rockets_title) + sum(num_rockets_body) as num_rockets,
            sum(num_diamonds_title) + sum(num_diamonds_body) as num_diamonds,
        from posts_enhanced
        group by author
    ),
    author_comments_stats as (
        select
            author,
            count(*) as num_comments,
            sum(num_words) as num_words,
            sum(num_rockets) as num_rockets,
            sum(num_diamonds) as num_diamonds
        from comments_enhanced
        group by author
    )
select 
    coalesce(p.author, c.author) as author,
    ifnull(p.num_posts, 0) as num_posts,
    ifnull(c.num_comments, 0) as num_comments,
    ifnull(p.num_words, 0) + ifnull(c.num_words, 0) as num_words,
    ifnull(p.num_rockets, 0) + ifnull(c.num_rockets, 0) as num_rockets,
    ifnull(p.num_diamonds, 0) + ifnull(c.num_diamonds, 0) as num_diamonds,
from author_posts_stats p
full join author_comments_stats c on p.author = c.author
""")

In [ ]:
authors_no_mods = authors.loc[[author not in MODERATORS for author in authors['author']]]

Author with the most posts

In [ ]:
authors_no_mods.sort_values('num_posts', ascending=False).head(3)

Author with the most comments

In [ ]:
authors_no_mods.sort_values('num_comments', ascending=False).head(3)

Author who wrote the most words

In [ ]:
authors_no_mods.sort_values('num_words', ascending=False).head(3)

Author who posted the most rockets

In [ ]:
authors_no_mods.sort_values('num_rockets', ascending=False).head(3)

Author who posted the most diamonds

In [ ]:
authors_no_mods.sort_values('num_diamonds', ascending=False).head(3)